In [1]:
import tensorflow as tf
import os

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

In [3]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

In [4]:
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [5]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [8]:
import numpy as np

In [7]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [9]:
tf_dataset = tf_dataset.filter(lambda x:x>0)
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
31


### Convert sales numbers from USA dollars ($) to Indian Rupees (PKR) Assuming 1->279 conversation rate

In [11]:
tf_dataset = tf_dataset.map(lambda x:x*279)
for sales in tf_dataset.take(3):
    print(sales.numpy())

5859
6138
8649


### Shuffle

In [13]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset:
    print(sales.numpy())

8649
8649
8928
6138
9486
5859


### Batching

In [14]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[6138 8649]
[8649 5859]
[9486 8928]


### Perform all of the above operations in one shot

In [15]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x:x>0).map(lambda y:y*279).shuffle(2).batch(2)

In [16]:
tf_dataset

<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>

In [17]:
for sales in tf_dataset:
    print(sales.numpy())

[6138 5859]
[8928 8649]
[9486 8649]


### Images

In [23]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

In [24]:
image_count = len(images_ds)
image_count

150

In [25]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [26]:
for file in images_ds.take(3):
    print(file.numpy())

b'images\\daisy\\102841525_bd6628ae3c.jpg'
b'images\\daisy\\1031799732_e7f4008c03.jpg'
b'images\\daisy\\105806915_a9c13e2106_n.jpg'


In [27]:
images_ds = images_ds.shuffle(200)

In [28]:
for file in images_ds.take(3):
    print(file.numpy())

b'images\\roses\\2682566502_967e7eaa2a.jpg'
b'images\\roses\\394990940_7af082cf8d_n.jpg'
b'images\\daisy\\105806915_a9c13e2106_n.jpg'


In [29]:
class_names = ["daisy", 'rose']

In [30]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [31]:
len(train_ds)

120

In [32]:
len(test_ds)

30

In [33]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [35]:
get_label(b'images\\roses\\2682566502_967e7eaa2a.jpg')

<tf.Tensor: shape=(), dtype=string, numpy=b'roses'>

In [39]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [40]:
img, label = process_image('images\\roses\\2682566502_967e7eaa2a.jpg')

In [41]:
img.numpy()[:2]

array([[[255.     , 255.     , 255.     ],
        [255.     , 255.     , 255.     ],
        [255.     , 255.     , 255.     ],
        [255.     , 255.     , 255.     ],
        [255.     , 255.     , 255.     ],
        [254.01129, 254.01129, 254.01129],
        [254.27734, 254.27734, 254.27734],
        [254.77899, 254.77899, 254.77899],
        [252.83203, 252.83203, 252.83203],
        [249.24377, 249.24377, 249.24377],
        [254.64996, 254.64996, 254.64996],
        [254.27734, 254.27734, 254.27734],
        [253.31757, 253.31757, 253.31757],
        [255.     , 255.     , 255.     ],
        [254.66003, 254.66003, 254.66003],
        [254.27734, 254.27734, 254.27734],
        [254.95312, 254.95312, 254.95312],
        [254.72266, 254.72266, 254.72266],
        [254.935  , 254.935  , 254.935  ],
        [255.     , 255.     , 255.     ],
        [254.58221, 254.58221, 254.58221],
        [254.62738, 254.62738, 254.62738],
        [255.     , 255.     , 255.     ],
        [25

In [42]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [43]:
for image, label in train_ds.take(1):
    print("****", image)
    print("****", label)

**** tf.Tensor(
[[[ 78.34418  134.45312   37.272583]
  [ 81.23175  133.61462   33.487   ]
  [ 82.46875  135.26562   35.      ]
  ...
  [ 77.21271   95.68146  112.94708 ]
  [ 77.76825   95.487    113.127625]
  [ 79.71051   97.10895  114.90973 ]]

 [[ 78.77997  132.77997   38.77997 ]
  [ 80.       134.40234   39.20703 ]
  [ 81.96344  134.75641   39.749695]
  ...
  [ 79.10687   97.57562  114.84125 ]
  [ 79.640625  97.359375 115.      ]
  [ 80.13153   97.13153  115.13153 ]]

 [[ 81.00391  130.90662   39.45526 ]
  [ 83.00391  133.9961    42.007812]
  [ 84.       135.        40.529175]
  ...
  [ 79.73334   97.26459  114.99896 ]
  [ 79.36328   97.64453  115.00391 ]
  [ 78.55078   97.55078  114.55078 ]]

 ...

 [[ 56.546875  81.90625   93.449585]
  [ 58.        85.71875   94.359375]
  [ 59.001038  86.99713   97.99713 ]
  ...
  [ 93.52344  114.52344  119.515625]
  [ 86.06891  110.71234  114.70813 ]
  [ 81.88281  106.44318  110.43927 ]]

 [[ 58.270813  82.27081   92.27081 ]
  [ 56.95703   86.214

In [44]:
def scale(image, label):
    return image/255, label

In [45]:
train_ds = train_ds.map(scale)

In [46]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.03193264 0.12212872 0.01232479]
****Label:  b'roses'
****Image:  [0.64150965 0.58679533 0.27266392]
****Label:  b'roses'
****Image:  [0.5863013  0.5548407  0.42365196]
****Label:  b'daisy'
****Image:  [0.8625766  0.71551776 0.5743413 ]
****Label:  b'roses'
****Image:  [0.19309226 0.49091366 0.11084439]
****Label:  b'daisy'
